In [1]:
%load_ext autoreload

%autoreload 1

In [2]:
import xarray as xr
from pathlib import Path
import numpy as np
import numexpr as ne

import time
from analysis_utils import load_data_set, calc_temp_spac_means

%aimport analysis_utils

In [3]:
data_path = Path(".").resolve().parent / "data/20220308T183847/OUTPUT/20220308T183847_3d.nc"
lad_path = Path(".").resolve().parent / "data/20220308T183847/INPUT/20220308T183847_static"
ds = load_data_set(data_path)
ds_lad = xr.open_dataset(lad_path)

In [4]:
dz = 3

z_scalar  = np.arange(150//dz, 300//dz)

urban_ratio = 0.5
y_domain = np.arange(int(ds.s.y.size * urban_ratio), ds.s.y.size)

In [9]:
ds_lad.isel(zlad=1).lad.values

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.12382875, 0.12342117, ..., 0.12331718, 0.12364402,
        0.        ],
       [0.        , 0.12355126, 0.12334539, ..., 0.12337404, 0.12362072,
        0.        ],
       [0.        , 0.12323531, 0.12320342, ..., 0.12325244, 0.12333724,
        0.        ]])

In [24]:
ds

<xarray.Dataset>
Dimensions:  (time: 72, x: 192, xu: 192, y: 432, yv: 432, zpc_3d: 6, zu_3d: 193, zw_3d: 193)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 562.5 565.5 568.5 571.5 574.5
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
  * yv       (yv) float64 0.0 3.0 6.0 9.0 ... 1.287e+03 1.29e+03 1.293e+03
  * zpc_3d   (zpc_3d) float64 0.0 1.5 4.5 7.5 10.5 13.5
    zu_3d_   (zu_3d) float64 1.5 4.5 7.5 10.5 13.5 ... 568.5 571.5 574.5 577.5
    zw_3d_   (zw_3d) float64 3.0 6.0 9.0 12.0 15.0 ... 570.0 573.0 576.0 579.0
Dimensions without coordinates: zu_3d, zw_3d
Data variables: (12/18)
    E_UTM    (x) float64 ...
    N_UTM    (y) float64 ...
    Eu_UTM   (xu) float64 ...
    Nu_UTM   (y) float64 ...
    Ev_UTM   (x) float64 ...
    Nv_UTM   (yv) float64 ...
    ...       ...
    crs      int32 ...
    theta    (time, zu_3d, y, x) float32 ...
    v        (time, zu_3d, yv, x) float32 1.232 1.322 1.4 ... 4.025 4.025 4.025
    w        (time, zw_3d, y, x) float32 0.2423 0.453 0.4481 ... 0.0 0.0 0.0
    u        (time, zu_3d, y, xu) float32 -0.01739 0.06189 ... 1.06e-05
    s        (time, zu_3d, y, x) float32 ...
Attributes: (12/27)
    title:           PALM 6.0  Rev: 4359  run: 20220308T183847.00  host: defa...
    Conventions:     CF-1.7
    creation_time:   2022-03-08 18:41:16 -05
    data_content:    3d
    version:         1
    origin_time:     2019-06-21 12:00:00 +00
    ...              ...
    source:          PALM 6.0 Rev: 4359
    references:      
    keywords:        
    licence:         
    comment:         
    VAR_LIST:        ;theta;v;w;u;s;

In [ ]:
s = time.time()
sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
ubar = (sub_ds.u**2 + sub_ds.v**2 + sub_ds.w**2)**(1/2)
print(time.time() - s)
# ubar_total = np.sqrt(np.square(ds.u.values) + np.square(ds.v.values) + np.square(ds.w.values))

In [5]:
s = time.time()
sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
# ubar = (sub_ds.u**2 + sub_ds.v**2 + sub_ds.w**2)**(1/2)

ubar_total = np.sqrt(np.square(sub_ds.u.values) + np.square(sub_ds.v.values) + np.square(sub_ds.w.values))
print(time.time() - s)

23.01164722442627


In [ ]:
s = time.time()
sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
# ubar = (sub_ds.u**2 + sub_ds.v**2 + sub_ds.w**2)**(1/2)

ubar_total = np.sqrt(np.square(sub_ds.u) + np.square(sub_ds.v) + np.square(sub_ds.w))
# ubar_total = np.sqrt(np.square(sub_ds.u.values) + np.square(sub_ds.v.values) + np.square(sub_ds.w.values))
print(time.time() - s)

In [7]:
s = time.time()
sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
u = sub_ds.u#.values
v = sub_ds.v#.values
w = sub_ds.w#.values
ubar = ne.evaluate("sqrt(u**2 + v**2 + w**2)")
print(time.time() - s)

11.071406841278076


In [5]:
ds.u

<xarray.DataArray 'u' (time: 72, zu_3d: 193, y: 432, xu: 192)>
[1152589824 values with dtype=float32]
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
    zu_3d_   (zu_3d) float64 1.5 4.5 7.5 10.5 13.5 ... 568.5 571.5 574.5 577.5
Dimensions without coordinates: zu_3d
Attributes:
    units:      m/s
    long_name:  u

In [11]:
TempSpacMeanU, TempSpacMeanV, TempSpacMeanW = calc_temp_spac_means(ds)

In [ ]:
uw = (ds.w - TempSpacMeanW) * (ds.u - TempSpacMeanU);
vw = (ds.w - TempSpacMeanW) * (ds.v - TempSpacMeanV);

In [ ]:
np.power(np.nanmean(uw)**2 + np.nanmean(vw)**2, 1/4)

In [13]:
ustar = (squeeze(nanmean(nanmean(uw,1),2).^2 + nanmean(nanmean(vw,1),2).^2)).^0.25;

KeyError: 'values'

In [8]:
ds.w

<xarray.DataArray 'w' (time: 72, zw_3d: 193, y: 432, x: 192)>
[1152589824 values with dtype=float32]
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 562.5 565.5 568.5 571.5 574.5
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
    zw_3d_   (zw_3d) float64 3.0 6.0 9.0 12.0 15.0 ... 570.0 573.0 576.0 579.0
Dimensions without coordinates: zw_3d
Attributes:
    units:      m/s
    long_name:  w

In [ ]:
ubar = ubar_total

In [23]:
type(ubar_total)

numpy.ndarray

In [7]:
ne.evaluate("exp(2)")

array(7.3890561)

In [21]:
scalar_grad = ds.s.isel(y=y_domain, zu_3d=z_scalar)

In [16]:
ds.s.y.size * urban_ratio

216.0

In [6]:
ds.s.isel(time=60, zu_3d=10).v

array([[0.1304376 , 0.1125954 , 0.09213795, ..., 0.0807151 , 0.14683247,
        0.14305805],
       [0.12404612, 0.1070879 , 0.07839098, ..., 0.08024671, 0.13751815,
        0.1399681 ],
       [0.12678802, 0.12038978, 0.10291763, ..., 0.08745276, 0.13599472,
        0.13938259],
       ...,
       [0.12819828, 0.14347515, 0.18452443, ..., 0.0778394 , 0.13670161,
        0.14400712],
       [0.13251767, 0.12401052, 0.14600988, ..., 0.07967169, 0.15714346,
        0.1510945 ],
       [0.13553588, 0.11933067, 0.11963697, ..., 0.082677  , 0.15785772,
        0.14794137]], dtype=float32)

In [38]:
da = xr.DataArray(np.arange(25).reshape(5, 5), dims=("x", "y"))
da

<xarray.DataArray (x: 5, y: 5)>
array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])
Dimensions without coordinates: x, y

In [37]:
tgt_x = xr.DataArray(np.arange(0, 5), dims="points")
tgt_y = xr.DataArray(np.arange(0, 5), dims="points")
da = da.isel(x=tgt_x, y=tgt_y)
da

ValueError: Dimensions {'y', 'x'} do not exist. Expected one or more of Frozen({'points': 5})

In [39]:
da.values

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [3]:
a = np.arange(25).reshape((5,5))
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])